In [3]:
#PRODUCT CREATION FROM WEB ORDER DEMO

import os
from dotenv import load_dotenv
import xmlrpc.client
from datetime import datetime

# Example payload for creating a sale order
payload = {
    "product": {
        "name": f"Finished Product {datetime.now().strftime('%Y%m%d_%H%M%S')}",
        "width": 500,
        "height": 500, 
        "price": 200.00,
        "reference": f"TEST_FINISHED_PRODUCT_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        "components": [
            {
                "name": "Frame A",
                "reference": "TEST_FRAME_A"
            },
            {
                "name": "Glass A", 
                "reference": "TEST_GLASS_A"
            },
            {
                "name": "Passe-Partout A",
                "reference": "TEST_PP_A"
            }
        ]
    }
}

# Reload environment variables
load_dotenv(override=True)

## Odoo connection details from environment variables
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB') 
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common')
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
models = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object')

# Create product
product_vals = {
    'name': payload['product']['name'],
    'type': 'consu',  # Changed from 'product' to 'consu' as it's a valid selection value
    'x_studio_width': payload['product']['width'],
    'x_studio_height': payload['product']['height'],
    'list_price': payload['product']['price'],
    'default_code': payload['product']['reference'],
    'categ_id': 8,  # Set category ID to 8
}

product_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.product', 'create', [product_vals])

# Get the product template ID from the created product
product_data = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.product', 'read', [product_id], {'fields': ['product_tmpl_id']}
)
product_tmpl_id = product_data[0]['product_tmpl_id'][0]

# Create components and BOM
bom_components = []
bom_operations = []
width = payload['product']['width']
height = payload['product']['height']
surface = width * height
circumference = 2 * (width + height)

print(f"Surface: {surface} mm²")
print(f"Surface: {surface/1000000} m²")
print(f"Circumference: {circumference} mm")

# Convert dimensions to meters for duration rules
surface_m2 = surface / 1000000  # Convert mm² to m²
circumference_m = circumference / 1000  # Convert mm to m

for component in payload['product']['components']:
    # Search for existing component
    component_ids = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'search',
        [[['default_code', '=', component['reference']]]])
    
    if not component_ids:
        # Create component if it doesn't exist
        component_vals = {
            'name': component['name'],
            'type': 'product',
            'default_code': component['reference'],
            'categ_id': 8,
        }
        component_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'product.product', 'create', [component_vals])
    else:
        component_id = component_ids[0]
    
    # Get component details to check price computation method and associated service
    component_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'read',
        [component_id],
        {'fields': ['x_studio_price_computation', 'x_studio_associated_service', 'x_studio_associated_service_duration_rule']})
    
    # Calculate quantity based on price computation method
    if component_info and component_info[0]['x_studio_price_computation'] == 'Circumference':
        quantity = circumference_m
    elif component_info and component_info[0]['x_studio_price_computation'] == 'Surface':
        quantity = surface_m2
    else:
        quantity = 1
    
    # Add to BOM components
    bom_components.append((0, 0, {
        'product_id': component_id,
        'product_qty': quantity,
    }))

    # Handle associated service/operation
    if component_info[0]['x_studio_associated_service']:
        service_id = component_info[0]['x_studio_associated_service'][0]
        
        # Get service details
        service_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'x_services', 'read',
            [service_id],
            {'fields': ['x_name', 'x_soort', 'x_studio_associated_work_center']})

        # Get duration rules directly from component
        duration_rules = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'x_services_duration_rules', 'read',
            [component_info[0]['x_studio_associated_service_duration_rule']],
            {'fields': ['x_omtrek', 'x_oppervlakte', 'x_duurtijd_totaal']})

        # Find appropriate duration based on x_soort
        if service_info[0]['x_soort'] == 'Oppervlakte':
            relevant_value = surface_m2  # Use m² value
            rules_sorted = sorted(duration_rules, key=lambda x: x['x_oppervlakte'])
            matching_rule = next((rule for rule in rules_sorted if rule['x_oppervlakte'] >= relevant_value), rules_sorted[-1])
            duration_seconds = matching_rule['x_duurtijd_totaal']
        else:  # 'Omtrek'
            relevant_value = circumference_m  # Use m value
            rules_sorted = sorted(duration_rules, key=lambda x: x['x_omtrek'])
            matching_rule = next((rule for rule in rules_sorted if rule['x_omtrek'] >= relevant_value), rules_sorted[-1])
            duration_seconds = matching_rule['x_duurtijd_totaal']

        # Convert duration from seconds to minutes and calculate MM:SS format
        duration_minutes = duration_seconds / 60
        minutes = int(duration_minutes)
        seconds = int((duration_minutes - minutes) * 60)
        odoo_display = f"{minutes:02d}:{seconds:02d}"
        print(f"Converting duration for {service_info[0]['x_name']}: {duration_seconds} seconds = {duration_minutes} minutes (will display as {odoo_display} in Odoo)")

        # Add operation to BOM
        bom_operations.append((0, 0, {
            'name': service_info[0]['x_name'],
            'time_cycle_manual': duration_minutes,
            'workcenter_id': service_info[0]['x_studio_associated_work_center'][0] if service_info[0]['x_studio_associated_work_center'] else False
        }))

# Create Bill of Materials using the template ID
bom_vals = {
    'product_tmpl_id': product_tmpl_id,
    'product_qty': 1,
    'type': 'normal',
    'bom_line_ids': bom_components,
    'operation_ids': bom_operations,
}

bom_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'mrp.bom', 'create', [bom_vals])

# Create sale order
order_vals = {
    'partner_id': 1,  # Default customer ID, adjust as needed
    'order_line': [(0, 0, {
        'product_id': product_id,
        'product_uom_qty': 1,
        'price_unit': payload['product']['price']
    })]
}

order_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'sale.order', 'create', [order_vals])

print(f"Created product ID: {product_id}")
print(f"Created BOM ID: {bom_id}")
print(f"Created sale order ID: {order_id}")
print("Sale order from web simulation finished")

Surface: 250000 mm²
Surface: 0.25 m²
Circumference: 2000 mm
Converting duration for Cutting Wood: 300 seconds = 5.0 minutes (will display as 05:00 in Odoo)
Converting duration for Cutting Glass: 620 seconds = 10.333333333333334 minutes (will display as 10:20 in Odoo)
Created product ID: 14810
Created BOM ID: 57
Created sale order ID: 50
Sale order from web simulation finished


In [5]:
#PRODUCT CREATION FROM EXISTING SALE ORDER (MANUAL PROCESS)

import os
from dotenv import load_dotenv
import xmlrpc.client
from datetime import datetime

# Reload environment variables
load_dotenv(override=True)

## Odoo connection details from environment variables
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB') 
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common')
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
models = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object')

# Get sale order
sale_order_id = 48
sale_order = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'sale.order', 'read',
    [sale_order_id],
    {'fields': ['order_line']})

# Get first product from sale order
order_line = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'sale.order.line', 'read',
    [sale_order[0]['order_line'][0]],
    {'fields': ['product_id', 'price_unit']})

product_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.product', 'read',
    [order_line[0]['product_id'][0]],
    {'fields': ['x_studio_width', 'x_studio_height', 'name', 'default_code']})

# Get product details
width = product_info[0]['x_studio_width']
height = product_info[0]['x_studio_height']
price = order_line[0]['price_unit']

# Get BOM for the product
bom_ids = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'mrp.bom', 'search',
    [[['product_tmpl_id', '=', product_info[0]['id']]]])

bom_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'mrp.bom', 'read',
    [bom_ids[0]],
    {'fields': ['bom_line_ids']})

# Get components from BOM
components = []
for bom_line_id in bom_info[0]['bom_line_ids']:
    bom_line = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'mrp.bom.line', 'read',
        [bom_line_id],
        {'fields': ['product_id']})
    component_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'read',
        [bom_line[0]['product_id'][0]],
        {'fields': ['name', 'default_code']})
    components.append({
        'name': component_info[0]['name'],
        'reference': component_info[0]['default_code']
    })

# Create new product with copied details
product_vals = {
    'name': f"Finished Product {datetime.now().strftime('%Y%m%d_%H%M%S')}",
    'type': 'consu',
    'x_studio_width': width,
    'x_studio_height': height,
    'list_price': price,
    'default_code': f"TEST_FINISHED_PRODUCT_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
    'categ_id': 8,
}

product_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.product', 'create', [product_vals])

# Create components and BOM
bom_components = []
bom_operations = []
surface = width * height
circumference = 2 * (width + height)

print(f"Surface: {surface} mm²")
print(f"Surface: {surface/1000000} m²")
print(f"Circumference: {circumference} mm")

# Convert dimensions to meters for duration rules
surface_m2 = surface / 1000000  # Convert mm² to m²
circumference_m = circumference / 1000  # Convert mm to m

for component in components:
    # Search for existing component
    component_ids = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'search',
        [[['default_code', '=', component['reference']]]])
    
    component_id = component_ids[0]
    
    # Get component details to check price computation method and associated service
    component_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'read',
        [component_id],
        {'fields': ['x_studio_price_computation', 'x_studio_associated_service', 'x_studio_associated_service_duration_rule']})
    
    # Calculate quantity based on price computation method
    if component_info and component_info[0]['x_studio_price_computation'] == 'Circumference':
        quantity = 2 * (width + height)
    elif component_info and component_info[0]['x_studio_price_computation'] == 'Surface':
        quantity = width * height
    else:
        quantity = 1
    
    # Add to BOM components
    bom_components.append((0, 0, {
        'product_id': component_id,
        'product_qty': quantity,
    }))

    # Handle associated service/operation
    if component_info[0]['x_studio_associated_service']:
        service_id = component_info[0]['x_studio_associated_service'][0]
        
        # Get service details
        service_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'x_services', 'read',
            [service_id],
            {'fields': ['x_name', 'x_soort', 'x_studio_associated_work_center']})

        # Get duration rules directly from component
        duration_rules = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'x_services_duration_rules', 'read',
            [component_info[0]['x_studio_associated_service_duration_rule']],
            {'fields': ['x_omtrek', 'x_oppervlakte', 'x_duurtijd_totaal']})

        # Find appropriate duration based on x_soort
        if service_info[0]['x_soort'] == 'Oppervlakte':
            relevant_value = surface_m2  # Use m² value
            rules_sorted = sorted(duration_rules, key=lambda x: x['x_oppervlakte'])
            matching_rule = next((rule for rule in rules_sorted if rule['x_oppervlakte'] >= relevant_value), rules_sorted[-1])
            duration_seconds = matching_rule['x_duurtijd_totaal']
        else:  # 'Omtrek'
            relevant_value = circumference_m  # Use m value
            rules_sorted = sorted(duration_rules, key=lambda x: x['x_omtrek'])
            matching_rule = next((rule for rule in rules_sorted if rule['x_omtrek'] >= relevant_value), rules_sorted[-1])
            duration_seconds = matching_rule['x_duurtijd_totaal']

        # Convert duration from seconds to minutes and calculate MM:SS format
        duration_minutes = duration_seconds / 60
        minutes = int(duration_minutes)
        seconds = int((duration_minutes - minutes) * 60)
        odoo_display = f"{minutes:02d}:{seconds:02d}"
        print(f"Converting duration for {service_info[0]['x_name']}: {duration_seconds} seconds = {duration_minutes} minutes (will display as {odoo_display} in Odoo)")

        # Add operation to BOM
        bom_operations.append((0, 0, {
            'name': service_info[0]['x_name'],
            'time_cycle_manual': duration_minutes,
            'workcenter_id': service_info[0]['x_studio_associated_work_center'][0] if service_info[0]['x_studio_associated_work_center'] else False
        }))

# Create Bill of Materials
bom_vals = {
    'product_tmpl_id': product_id,
    'product_qty': 1,
    'type': 'normal',
    'bom_line_ids': bom_components,
    'operation_ids': bom_operations
}

bom_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'mrp.bom', 'create', [bom_vals])

# Update existing sale order with new product
models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'sale.order', 'write',
    [sale_order_id, {
        'order_line': [(1, sale_order[0]['order_line'][0], {
            'product_id': product_id,
            'product_uom_qty': 1,
            'price_unit': price
        })]
    }])

print(f"Created product ID: {product_id}")
print(f"Created BOM ID: {bom_id}")
print(f"Updated sale order ID: {sale_order_id}")
print("Manual process simulation finished")

Surface: 10000.0 mm²
Surface: 0.01 m²
Circumference: 400.0 mm
Converting duration for Cutting Wood: 300 seconds = 5.0 minutes (will display as 05:00 in Odoo)
Converting duration for Cutting Glass: 620 seconds = 10.333333333333334 minutes (will display as 10:20 in Odoo)
Created product ID: 73
Created BOM ID: 53
Updated sale order ID: 48
Manual process simulation finished


In [ ]:
#PRODUCT CREATION DEMO (OLD, before operations)


import os
from dotenv import load_dotenv
import xmlrpc.client
from datetime import datetime

# Example payload for creating a sale order
payload = {
    "product": {
        "name": f"Finished Product {datetime.now().strftime('%Y%m%d_%H%M%S')}",
        "width": 50,
        "height": 100, 
        "price": 200.00,
        "reference": f"TEST_FINISHED_PRODUCT_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        "components": [
            {
                "name": "Frame A",
                "reference": "TEST_FRAME_B"
            },
            {
                "name": "Glass A", 
                "reference": "TEST_GLASS_A"
            },
            {
                "name": "Passe-Partout A",
                "reference": "TEST_PP_A"
            }
        ]
    }
}


# Reload environment variables
load_dotenv(override=True)

## Odoo connection details from environment variables
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB') 
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common')
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
models = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object')

# Create product
product_vals = {
    'name': payload['product']['name'],
    'type': 'consu',  # Changed from 'product' to 'consu' as it's a valid selection value
    'x_studio_width': payload['product']['width'],
    'x_studio_height': payload['product']['height'],
    'list_price': payload['product']['price'],
    'default_code': payload['product']['reference'],
    'categ_id': 8,  # Set category ID to 8
}

product_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.product', 'create', [product_vals])

# Create components and BOM
bom_components = []
width = payload['product']['width']
height = payload['product']['height']

for component in payload['product']['components']:
    # Search for existing component
    component_ids = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'search',
        [[['default_code', '=', component['reference']]]])
    
    if not component_ids:
        # Create component if it doesn't exist
        component_vals = {
            'name': component['name'],
            'type': 'product',
            'default_code': component['reference'],
            'categ_id': 8,
        }
        component_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'product.product', 'create', [component_vals])
    else:
        component_id = component_ids[0]
    
    # Get component details to check price computation method
    component_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'read',
        [component_id],
        {'fields': ['x_studio_price_computation']})
    
    # Calculate quantity based on price computation method
    if component_info and component_info[0]['x_studio_price_computation'] == 'Circumference':
        quantity = 2 * (width + height)
    elif component_info and component_info[0]['x_studio_price_computation'] == 'Surface':
        quantity = width * height
    else:
        quantity = 1
    
    # Add to BOM components
    bom_components.append((0, 0, {
        'product_id': component_id,
        'product_qty': quantity,
    }))

# Create Bill of Materials
bom_vals = {
    'product_tmpl_id': product_id,
    'product_qty': 1,
    'type': 'normal',
    'bom_line_ids': bom_components
}

bom_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'mrp.bom', 'create', [bom_vals])

# Create sale order
order_vals = {
    'partner_id': 1,  # Default customer ID, adjust as needed
    'order_line': [(0, 0, {
        'product_id': product_id,
        'product_uom_qty': 1,
        'price_unit': payload['product']['price']
    })]
}

order_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'sale.order', 'create', [order_vals])

print(f"Created product ID: {product_id}")
print(f"Created BOM ID: {bom_id}")
print(f"Created sale order ID: {order_id}")

In [ ]:
#FETCHING SUPPLIER INFO TO MANAGE SEQUENCE

import os
from dotenv import load_dotenv
import xmlrpc.client

# Reload environment variables
load_dotenv(override=True)

## Odoo connection details from environment variables
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB') 
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common')
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

# Create object proxy
models = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object')

# Fetch product with ID 4 and get seller_ids field
product = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.template', 'read',
    [4],
    {'fields': ['seller_ids']}
)

if product:
    seller_ids = product[0]['seller_ids']
    print("Seller IDs:", seller_ids)
    
    # Fetch detailed supplier info for each seller_id
    supplier_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.supplierinfo', 'read',
        [seller_ids],
        {'fields': ['id', 'display_name', 'sequence', 'product_tmpl_id']}
    )
    print("\nDetailed Supplier Information:")
    for info in supplier_info:
        print(info)
else:
    print("Product not found")

Seller IDs: [1, 4, 2]

Detailed Supplier Information:
{'id': 1, 'display_name': 'Louis Test (0.0 Units - 15.00\xa0€)', 'sequence': 1, 'product_tmpl_id': [4, 'Frame A']}
{'id': 4, 'display_name': 'Louis Test 3 (0.0 Units - 19.00\xa0€)', 'sequence': 2, 'product_tmpl_id': [4, 'Frame A']}
{'id': 2, 'display_name': 'Louis Test 2 (0.0 Units - 6.00\xa0€)', 'sequence': 3, 'product_tmpl_id': [4, 'Frame A']}


In [ ]:
#EXPORTING SPREADSHEET FINAL APPROACH
#(fetching spreadsheet.dashboard not working as it only get the odoo. formula, without the associated value)
#Using spreadsheet.dashboard.share is a workaround

import os
import json
import base64
import xmlrpc.client
from dotenv import load_dotenv
import xlsxwriter
import openpyxl

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object")

# Step 1: Get spreadsheet JSON (binary field)
spreadsheets = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB,
    uid,
    JUSTFRAMEIT_ODOO_API_KEY,
    'spreadsheet.dashboard.share',
    'search_read',
    [[('display_name', '=', 'Price List 1 (copy 4)')]],
    {'fields': ['id', 'display_name', 'spreadsheet_data'], 'limit': 1, 'order': 'create_date desc'}
)

if not spreadsheets:
    raise Exception("❌ No spreadsheet found with name 'Price List 1'.")

spreadsheet = spreadsheets[0]
print(f"✅ Found spreadsheet: {spreadsheet['display_name']} (ID {spreadsheet['id']})")

# Parse the spreadsheet data
data = json.loads(spreadsheet['spreadsheet_data'])

# Create workbook
workbook = xlsxwriter.Workbook('pricelist5.xlsx')

for sheet in data['sheets']:
    worksheet = workbook.add_worksheet(sheet['name'])
    worksheet.use_future_functions = True
    
    # Set values
    for cell_ref, value in sheet['cells'].items():
        col = openpyxl.utils.column_index_from_string(''.join(filter(str.isalpha, cell_ref))) - 1
        row = int(''.join(filter(str.isdigit, cell_ref))) - 1
        
        # Convert string numbers to float if possible
        if isinstance(value, str):
            try:
                value = float(value)
            except ValueError:
                pass
                
        worksheet.write(row, col, value)

workbook.close()
print(f"✅ Spreadsheet exported successfully to pricelist5.xlsx")


✅ Found spreadsheet: Price List 1 (copy 4) (ID 41)
✅ Formula written to cell N6
✅ Formula written to cell O6
✅ Formula written to cell P6
✅ Formula written to cell Q6
✅ Formula written to cell R6
✅ Formula written to cell S6
✅ Formula written to cell T6
✅ Formula written to cell U6
✅ Formula written to cell V6
✅ Formula written to cell W6
✅ Formula written to cell X6
✅ Formula written to cell Y6
✅ Formula written to cell Z6
✅ Formula written to cell AA6
✅ Formula written to cell AB6
✅ Formula written to cell AC6
✅ Formula written to cell AD6
✅ Formula written to cell AE6
✅ Formula written to cell AF6
✅ Formula written to cell AG6
✅ Formula written to cell AH6
✅ Formula written to cell AI6
✅ Formula written to cell AJ6
✅ Formula written to cell AK6
✅ Formula written to cell AL6
✅ Formula written to cell AM6
✅ Formula written to cell AN6
✅ Formula written to cell AO6
✅ Formula written to cell AP6
✅ Formula written to cell AQ6
✅ Formula written to cell AR6
✅ Formula written to cell AS6
✅ 

In [23]:
#EXPORTING SPREADSHEET FINAL APPROACH WITH N6 EDIT
#(fetching spreadsheet.dashboard not working as it only get the odoo. formula, without the associated value)
#Using spreadsheet.dashboard.share is a workaround

import os
import json
import base64
import xmlrpc.client
from dotenv import load_dotenv
import xlsxwriter
import openpyxl
import time

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object")

# Step 1: Get spreadsheet JSON (binary field)
spreadsheets = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB,
    uid,
    JUSTFRAMEIT_ODOO_API_KEY,
    'spreadsheet.dashboard.share',
    'search_read',
    [[('display_name', '=', 'Price List 1 (to share)')]],
    {'fields': ['id', 'display_name', 'spreadsheet_data'], 'limit': 1, 'order': 'create_date desc'}
)

if not spreadsheets:
    raise Exception("❌ No spreadsheet found with name 'Price List 1 (to share)'.")

spreadsheet = spreadsheets[0]
print(f"✅ Found spreadsheet: {spreadsheet['display_name']} (ID {spreadsheet['id']})")

# Parse the spreadsheet data
data = json.loads(spreadsheet['spreadsheet_data'])

# Create workbook
workbook = xlsxwriter.Workbook('pricelist5.xlsx')

for sheet_index, sheet in enumerate(data['sheets']):
    worksheet = workbook.add_worksheet(sheet['name'])
    worksheet.use_future_functions = True
    
    # Set values
    for cell_ref, value in sheet['cells'].items():
        col = openpyxl.utils.column_index_from_string(''.join(filter(str.isalpha, cell_ref))) - 1
        row = int(''.join(filter(str.isdigit, cell_ref))) - 1
        
        # Convert string numbers to float if possible
        if isinstance(value, str):
            try:
                value = float(value)
            except ValueError:
                pass
                
        worksheet.write(row, col, value)
    
    # Add formula to N6:FF2000 only for the first sheet
    if sheet_index == 0:
        formula = '=(IF($H6=$M$4,N$4*$I6,N$3*$I6)+(INDEX(FILTER(\'Service Duration\'!$D$1:$D$400,(\'Service Duration\'!$A$1:$A$400=$J6)*(\'Service Duration\'!$C$1:$C$400=MINIFS(\'Service Duration\'!$C$1:$C$400,\'Service Duration\'!$A$1:$A$400,$J6,\'Service Duration\'!$C$1:$C$400,">="&IF($H6=$M$4,N$4,N$3)))),1))*$L6/3600)*(1+$M6)'
        
        # N is column 13 (0-indexed), FF is column 161
        start_col = 13  # N
        end_col = 161   # FF
        start_row = 5   # Row 6 (0-indexed)
        end_row = 1999  # Row 2000 (0-indexed)
        
        total_rows = end_row - start_row + 1
        start_time = time.time()
        
        for row in range(start_row, end_row + 1):
            for col in range(start_col, end_col + 1):
                # Adjust formula for each row and column
                col_letter = openpyxl.utils.get_column_letter(col + 1)
                
                # Log check for each column to determine when to display dimension
                if row == start_row:  # Only log once per column (first row)
                    print(f"📊 Column {col_letter} (index {col}): Processing formulas for rows {start_row+1} to {end_row+1}")
                
                adjusted_formula = formula.replace('N$4', f'{col_letter}$4').replace('N$3', f'{col_letter}$3').replace('$H6', f'$H{row+1}').replace('$I6', f'$I{row+1}').replace('$J6', f'$J{row+1}').replace('$L6', f'$L{row+1}').replace('$M6', f'$M{row+1}')
                worksheet.write_formula(row, col, adjusted_formula)
            
            # Log every 10 rows with estimated time remaining
            if (row - start_row + 1) % 10 == 0:
                rows_processed = row - start_row + 1
                elapsed_time = time.time() - start_time
                avg_time_per_row = elapsed_time / rows_processed
                rows_remaining = total_rows - rows_processed
                estimated_time_remaining = avg_time_per_row * rows_remaining
                
                # Format time as minutes and seconds
                minutes = int(estimated_time_remaining // 60)
                seconds = int(estimated_time_remaining % 60)
                time_str = f"{minutes}m {seconds}s"
                
                print(f"✅ Processed {rows_processed}/{total_rows} rows (up to row {row+1}) - Estimated time remaining: {time_str}")

workbook.close()
print(f"✅ Spreadsheet exported successfully to pricelist5.xlsx")



✅ Found spreadsheet: Price List 1 (to share) (ID 42)
📊 Column N (index 13): Processing formulas for rows 6 to 2000
📊 Column O (index 14): Processing formulas for rows 6 to 2000
📊 Column P (index 15): Processing formulas for rows 6 to 2000
📊 Column Q (index 16): Processing formulas for rows 6 to 2000
📊 Column R (index 17): Processing formulas for rows 6 to 2000
📊 Column S (index 18): Processing formulas for rows 6 to 2000
📊 Column T (index 19): Processing formulas for rows 6 to 2000
📊 Column U (index 20): Processing formulas for rows 6 to 2000
📊 Column V (index 21): Processing formulas for rows 6 to 2000
📊 Column W (index 22): Processing formulas for rows 6 to 2000
📊 Column X (index 23): Processing formulas for rows 6 to 2000
📊 Column Y (index 24): Processing formulas for rows 6 to 2000
📊 Column Z (index 25): Processing formulas for rows 6 to 2000
📊 Column AA (index 26): Processing formulas for rows 6 to 2000
📊 Column AB (index 27): Processing formulas for rows 6 to 2000
📊 Column AC (in

KeyboardInterrupt: 

In [37]:
#DELETING SHARED SPREADSHEETS

import os
import json
import base64
import xmlrpc.client
from dotenv import load_dotenv
from openpyxl import Workbook

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object")

# Step 1: Search for all Price List 1 spreadsheets
spreadsheets = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB,
    uid,
    JUSTFRAMEIT_ODOO_API_KEY,
    'spreadsheet.dashboard.share',
    'search_read',
    [[('display_name', '=', 'Price List 1 (copy 4)')]],
    {'fields': ['id']}
)

if not spreadsheets:
    print("No spreadsheets found with name 'Price List 1 (copy 4)'")
else:
    # Get all IDs
    spreadsheet_ids = [s['id'] for s in spreadsheets]
    print(f"Found {len(spreadsheet_ids)} spreadsheets to delete")
    
    # Delete all found spreadsheets
    try:
        models.execute_kw(
            JUSTFRAMEIT_ODOO_DB,
            uid,
            JUSTFRAMEIT_ODOO_API_KEY,
            'spreadsheet.dashboard.share',
            'unlink',
            [spreadsheet_ids]
        )
        print(f"✅ Successfully deleted {len(spreadsheet_ids)} spreadsheets")
    except Exception as e:
        print(f"❌ Failed to delete spreadsheets: {str(e)}")


Found 2 spreadsheets to delete
✅ Successfully deleted 2 spreadsheets


In [ ]:
#COMPUTE PRICE FROM BOM

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo with allow_none=True to handle None values
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)

# Get initial cost before computing BOM
product_id = 60
initial_cost = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB,
    uid,
    JUSTFRAMEIT_ODOO_API_KEY,
    'product.template',
    'read',
    [[product_id]],
    {'fields': ['standard_price']}
)[0]['standard_price']

print(f"Initial cost: {initial_cost}")

# Compute BOM cost for product template ID 60
try:
    # Ignore return value from button_bom_cost
    models.execute_kw(
        JUSTFRAMEIT_ODOO_DB,
        uid,
        JUSTFRAMEIT_ODOO_API_KEY,
        'product.template',
        'button_bom_cost',
        [[product_id]]
    )
    
    # Get new cost after computation
    new_cost = models.execute_kw(
        JUSTFRAMEIT_ODOO_DB,
        uid,
        JUSTFRAMEIT_ODOO_API_KEY,
        'product.template',
        'read',
        [[product_id]],
        {'fields': ['standard_price']}
    )[0]['standard_price']
    
    print(f"New cost: {new_cost}")
    
    if new_cost != initial_cost:
        print("✅ Cost change detected - BOM computation successful")
    else:
        print("⚠️ Warning: No cost change detected after BOM computation")
        
except:
    # Silently continue if button_bom_cost returns None
    # Get new cost after computation
    new_cost = models.execute_kw(
        JUSTFRAMEIT_ODOO_DB,
        uid,
        JUSTFRAMEIT_ODOO_API_KEY,
        'product.template',
        'read',
        [[product_id]],
        {'fields': ['standard_price']}
    )[0]['standard_price']
    
    print(f"New cost: {new_cost}")
    
    if new_cost != initial_cost:
        print("✅ Cost change detected - BOM computation successful")
    else:
        print("⚠️ Warning: No cost change detected after BOM computation")

Initial cost: 1000.0
New cost: 1334417.22
✅ Cost change detected - BOM computation successful


In [6]:
import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)

sale_order_id = 55

try:
    # Trigger "Update Price (based on price list)"
    result = models.execute_kw(
        JUSTFRAMEIT_ODOO_DB,
        uid,
        JUSTFRAMEIT_ODOO_API_KEY,
        'sale.order',
        'action_update_prices',  # ✅ correct method name
        [[sale_order_id]]
    )
    print(f"✅ Successfully triggered 'Update Price (based on price list)' for Sale Order ID {sale_order_id}")
    if result:
        print("Returned value:", result)

except Exception as e:
    print(f"❌ Failed to trigger price update: {e}")


❌ Failed to trigger price update: <Fault 1: 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/saas-18.4/addons/rpc/controllers.py", line 164, in xmlrpc_2\n    response = self._xmlrpc(service)\n               ^^^^^^^^^^^^^^^^^^^^^\n  File "/home/odoo/src/custom/trial/saas_trial/controllers/main.py", line 442, in _xmlrpc\n    res = super()._xmlrpc(service)\n          ^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/home/odoo/src/odoo/saas-18.4/addons/rpc/controllers.py", line 139, in _xmlrpc\n    return dumps((result,))\n           ^^^^^^^^^^^^^^^^\n  File "/home/odoo/src/odoo/saas-18.4/addons/rpc/controllers.py", line 112, in dumps\n    response = OdooMarshaller(allow_none=False).dumps(params)\n               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/usr/lib/python3.12/xmlrpc/client.py", line 514, in dumps\n    dump(v, write)\n  File "/usr/lib/python3.12/xmlrpc/client.py", line 536, in __dump\n    f(self, value, write)\n  File "/usr/lib/python3.12/xmlrpc/client.py", line

In [ ]:
import os
import xmlrpc.client
from dotenv import load_dotenv

# =============================================
# 🔧 Load environment variables
# =============================================
load_dotenv(override=True)

ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# =============================================
# 🔗 Connect to Odoo
# =============================================
print("Connecting to Odoo...")

common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common", allow_none=True)
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

print(f"✅ Connected as {ODOO_USERNAME} (uid={uid})")

models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object", allow_none=True)

# =============================================
# 📦 Fetch all products (optimized)
# =============================================
print("\n🔍 Fetching all products...")

try:
    # Get total count first
    product_count = models.execute_kw(
        ODOO_DB, uid, ODOO_API_KEY,
        'product.product', 'search_count',
        [[]]  # Empty domain = all products
    )
    
    print(f"📊 Total products found: {product_count}")
    
    # Fetch only the 'name' field for all products (optimized query)
    products = models.execute_kw(
        ODOO_DB, uid, ODOO_API_KEY,
        'product.product', 'search_read',
        [[]],  # Empty domain = all products
        {'fields': ['name']}  # Only fetch name field
    )
    
    print(f"\n📋 Product Names ({len(products)} products):")
    print("="*60)
    
    for idx, product in enumerate(products, 1):
        print(f"#{idx:4d} - {product.get('name')}")
    
    print("="*60)
    print(f"✅ Displayed {len(products)} products")
    
except Exception as e:
    print(f"❌ Failed to fetch products: {e}")
    import traceback
    traceback.print_exc()

print("\n✅ Done!")


Connecting to Odoo...
✅ Connected as louisdresse@gmail.com (uid=2)

🔍 Fetching all products...
📊 Total products found: 5031

📋 Product Names (5031 products):
#   1 - Noten Gouden Bies 1,5 cm
#   2 - Notelaar Antiek Gouden Bies 1,5 Cm
#   3 - '' Metall '' Wit 2 Cm
#   4 - Copy of Template A_20250923_224107
#   5 - Finished Product 20250923_224630
#   6 - Standard delivery
#   7 - Finished Product 20251029_181053
#   8 - Finished Product 20251029_182527
#   9 - Finished Product 20251029_182622
#  10 - Finished Product 20251029_184738
#  11 - Finished Product 20251029_185015
#  12 - Finished Product 20251029_185627
#  13 - Finished Product 20251029_185746
#  14 - Finished Product 20251029_191224
#  15 - Finished Product 20251029_191251
#  16 - Finished Product 20251029_191321
#  17 - Finished Product 20251029_191340
#  18 - Finished Product 20251029_191623
#  19 - Finished Product 20251029_191707
#  20 - Finished Product 20251029_210626
#  21 - Finished Product 20251029_210720
#  22 - Fin

: 

In [13]:
import os
import xmlrpc.client
from dotenv import load_dotenv
from openpyxl import load_workbook
import time
from datetime import datetime, timedelta
import shutil

load_dotenv(override=True)

ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

print("Connecting to Odoo...")

common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common", allow_none=True)
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

print(f"✅ Connected as {ODOO_USERNAME} (uid={uid})")

models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object", allow_none=True)

print("\n🔍 Fetching products with price computation = Surface or Circumference...")

try:
    # Fetch ALL products with price computation = Surface or Circumference
    products = models.execute_kw(
        ODOO_DB, uid, ODOO_API_KEY,
        'product.product', 'search_read',
        [[['x_studio_price_computation', 'in', ['Surface', 'Circumference']]]],  # Filter by price computation
        {
            'fields': [
                'name', 
                'id', 
                'x_studio_product_code', 
                'x_studio_location_code', 
                'description_ecommerce', 
                'x_studio_price_computation', 
                'standard_price',
                'x_studio_associated_service', 
                'x_studio_associated_work_center', 
                'x_studio_associated_cost_per_employee_per_hour'
            ]
            # No limit - fetch all products
        }
    )
    
    total_products = len(products)
    print(f"📊 Fetched {total_products} products")
    
    # Create a new filename with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    template_file = 'justframeit pricelist template empty.xlsx'
    output_file = f'justframeit_pricelist_{timestamp}.xlsx'
    
    # Copy the template to create a new file
    shutil.copy(template_file, output_file)
    print(f"📄 Created new file: {output_file}")
    
    # Load the newly created Excel file
    wb = load_workbook(output_file)
    
    # =============================================
    # 📋 Fill TAB 1 with products
    # =============================================
    ws1 = wb.worksheets[0]  # First tab (index 0)
    
    # Starting row is A7 (row 7)
    start_row = 7
    start_time = time.time()
    
    # Fill data for each product
    for idx, product in enumerate(products):
        current_row = start_row + idx
        
        # Helper function to convert list/tuple values to string
        def convert_value(value):
            if isinstance(value, (list, tuple)):
                # If it's a list/tuple, join elements or take the second element (name)
                if len(value) > 1:
                    return str(value[1])  # Return the name part
                return str(value[0]) if value else ''
            return value
        
        # Fill columns A through J with the product data
        ws1[f'A{current_row}'] = convert_value(product.get('name'))
        ws1[f'B{current_row}'] = convert_value(product.get('id'))
        ws1[f'C{current_row}'] = convert_value(product.get('x_studio_product_code'))
        ws1[f'D{current_row}'] = convert_value(product.get('x_studio_location_code'))
        ws1[f'E{current_row}'] = convert_value(product.get('description_ecommerce'))
        ws1[f'F{current_row}'] = convert_value(product.get('x_studio_price_computation'))
        ws1[f'G{current_row}'] = convert_value(product.get('standard_price'))
        ws1[f'H{current_row}'] = convert_value(product.get('x_studio_associated_service'))
        ws1[f'I{current_row}'] = convert_value(product.get('x_studio_associated_work_center'))
        ws1[f'J{current_row}'] = convert_value(product.get('x_studio_associated_cost_per_employee_per_hour'))
        
        print(f"✅ Filled row {current_row} on tab 1 with product: {product.get('name')}")
    
    # Save the workbook
    wb.save(output_file)
    print(f"\n✅ Successfully filled {len(products)} products into tab 1")
    
    # =============================================
    # 📋 Fetch pricelists and fill TAB 2
    # =============================================
    print("\n🔍 Fetching pricelists...")
    
    pricelists = models.execute_kw(
        ODOO_DB, uid, ODOO_API_KEY,
        'product.pricelist', 'search_read',
        [[]],  # Empty domain = all pricelists
        {
            'fields': ['name', 'x_studio_price_discount']
        }
    )
    
    print(f"📊 Fetched {len(pricelists)} pricelists")
    
    # Access the second tab (sheet)
    if len(wb.sheetnames) > 1:
        ws2 = wb.worksheets[1]
    else:
        print("⚠️ Warning: Second tab not found in workbook")
        ws2 = None
    
    if ws2:
        # Starting row is A3 (row 3)
        pricelist_start_row = 3
        
        # Fill data for each pricelist
        for idx, pricelist in enumerate(pricelists):
            current_row = pricelist_start_row + idx
            
            # Helper function to convert list/tuple values to string
            def convert_value(value):
                if isinstance(value, (list, tuple)):
                    if len(value) > 1:
                        return str(value[1])
                    return str(value[0]) if value else ''
                return value
            
            # Fill columns A and B with pricelist data
            ws2[f'A{current_row}'] = convert_value(pricelist.get('name'))
            ws2[f'B{current_row}'] = convert_value(pricelist.get('x_studio_price_discount'))
            
            print(f"✅ Filled row {current_row} on tab 2 with pricelist: {pricelist.get('name')}")
        
        # Save the workbook again with pricelist data
        wb.save(output_file)
        print(f"\n✅ Successfully filled {len(pricelists)} pricelists into tab 2")
    
    # =============================================
    # 📋 Fetch service duration rules and fill TAB 3
    # =============================================
    print("\n🔍 Fetching service duration rules...")
    
    duration_rules = models.execute_kw(
        ODOO_DB, uid, ODOO_API_KEY,
        'x_services_duration_rules', 'search_read',
        [[]],  # Empty domain = all duration rules
        {
            'fields': [
                'x_associated_service',
                'x_studio_work_center',
                'x_studio_quantity',
                'x_duurtijd_totaal'
            ]
        }
    )
    
    print(f"📊 Fetched {len(duration_rules)} service duration rules")
    
    # Access the third tab (sheet)
    if len(wb.sheetnames) > 2:
        ws3 = wb.worksheets[2]
    else:
        print("⚠️ Warning: Third tab not found in workbook")
        ws3 = None
    
    if ws3:
        # Starting row is A3 (row 3)
        duration_start_row = 3
        
        # Fill data for each duration rule
        for idx, rule in enumerate(duration_rules):
            current_row = duration_start_row + idx
            
            # Helper function to convert list/tuple values to string
            def convert_value(value):
                if isinstance(value, (list, tuple)):
                    if len(value) > 1:
                        return str(value[1])
                    return str(value[0]) if value else ''
                return value
            
            # Fill columns A through D with duration rule data
            ws3[f'A{current_row}'] = convert_value(rule.get('x_associated_service'))
            ws3[f'B{current_row}'] = convert_value(rule.get('x_studio_work_center'))
            ws3[f'C{current_row}'] = convert_value(rule.get('x_studio_quantity'))
            ws3[f'D{current_row}'] = convert_value(rule.get('x_duurtijd_totaal'))
            
            print(f"✅ Filled row {current_row} on tab 3 with service: {rule.get('x_associated_service')}")
        
        # Save the workbook again with duration rules data
        wb.save(output_file)
        print(f"\n✅ Successfully filled {len(duration_rules)} service duration rules into tab 3")
    
except Exception as e:
    print(f"❌ Failed to fetch products or fill template: {e}")
    import traceback
    traceback.print_exc()

print("\n✅ Done!")


Connecting to Odoo...
✅ Connected as louisdresse@gmail.com (uid=2)

🔍 Fetching products with price computation = Surface or Circumference...
📊 Fetched 1893 products
📄 Created new file: justframeit_pricelist_20251113_184919.xlsx
✅ Filled row 7 on tab 1 with product: Noten Gouden Bies 1,5 cm
✅ Filled row 8 on tab 1 with product: Notelaar Antiek Gouden Bies 1,5 Cm
✅ Filled row 9 on tab 1 with product: '' Metall '' Wit 2 Cm
✅ Filled row 10 on tab 1 with product: White              
✅ Filled row 11 on tab 1 with product: MDF 3MM BRUT Verpakking
✅ Filled row 12 on tab 1 with product: Mat plexiglas
✅ Filled row 13 on tab 1 with product: Frame A
✅ Filled row 14 on tab 1 with product: Frame B
✅ Filled row 15 on tab 1 with product: Frame C
✅ Filled row 16 on tab 1 with product: Glass A
✅ Filled row 17 on tab 1 with product: Glass B
✅ Filled row 18 on tab 1 with product: Passe-Partout A
✅ Filled row 19 on tab 1 with product: Passe-Partout B
✅ Filled row 20 on tab 1 with product:  "COLORBOX" BAKLI

In [ ]:
#CREATE PRODUCT PRICE EXCEL

import pandas as pd
import openpyxl
from datetime import datetime
import re

# Load the Excel file
excel_file = "justframeit_pricelist_20251113_184919.xlsx"
print(f"📂 Loading Excel file: {excel_file}")
wb = openpyxl.load_workbook(excel_file, data_only=True)  # data_only=True to get calculated values
print(f"✅ Excel file loaded successfully")

# Access the first tab
ws = wb.worksheets[0]
print(f"📄 Accessing first worksheet: {ws.title}")

# Headers are in row 6
header_row = 6
start_row = 7  # Data starts from row 7

# Find the last row with data in column A (starting from row 7)
last_row = start_row

print(f"🔍 Finding last row with data...")
# Find where column A becomes 0 or empty
for row in range(start_row, ws.max_row + 1):
    if row % 100 == 0:  # Log progress every 100 rows
        print(f"   Checking row {row}...")
    cell_value = ws[f'A{row}'].value
    if cell_value == 0 or cell_value is None or cell_value == '':
        break
    last_row = row

print(f"📊 Extracting data from row {start_row} to row {last_row}")

# Get headers from row 6, but check row 5 for dimension indicators
print(f"📋 Reading headers from row {header_row}...")
headers = []
found_first_dimension = False
last_dimension_col = 0
for col in range(1, ws.max_column + 1):
    if col % 50 == 0:  # Log progress every 50 columns
        print(f"   Processing column {col}...")
    # Check row 5 for dimension indicator (e.g., "10.0 x 20.0")
    dimension_cell = ws.cell(row=5, column=col)
    dimension_value = dimension_cell.value
    
    # Get the header from row 6
    header_cell = ws.cell(row=header_row, column=col)
    header_value = header_cell.value
    
    # Stricter check: dimension must contain 'x' AND at least one number
    is_dimension = False
    if dimension_value and isinstance(dimension_value, str):
        # Check if it contains 'x' and at least one digit
        if 'x' in dimension_value.lower() and re.search(r'\d', dimension_value):
            is_dimension = True
            found_first_dimension = True
            last_dimension_col = col
    
    # If we found dimensions before but now stopped finding them, stop processing columns
    if found_first_dimension and not is_dimension and not header_value:
        print(f"🛑 Stopping at column {col} - no more dimension headers found")
        break
    
    # If row 5 contains a valid dimension indicator, use it
    if is_dimension:
        headers.append(dimension_value)
    elif header_value:
        headers.append(header_value)
    else:
        headers.append(f'Column_{col}')

print(f"✅ Found {len(headers)} columns (last dimension column: {last_dimension_col})")

# Extract data from row 7 onwards to the last valid row
print(f"📥 Extracting data from {last_row - start_row + 1} rows...")
data = []
for row in range(start_row, last_row + 1):
    if (row - start_row) % 50 == 0:  # Log progress every 50 rows
        print(f"   Processing row {row} ({row - start_row + 1}/{last_row - start_row + 1})...")
    row_data = []
    for col in range(1, len(headers) + 1):
        cell = ws.cell(row=row, column=col)
        # Get the actual value, not the formula
        cell_value = cell.value
        
        # Handle special cases
        if cell_value is None:
            row_data.append(None)
        elif isinstance(cell_value, str) and cell_value.startswith('!REF'):
            # Handle reference errors
            row_data.append(None)
        else:
            row_data.append(cell_value)
    data.append(row_data)

# Create DataFrame
print(f"🔄 Creating DataFrame...")
df = pd.DataFrame(data, columns=headers)
print(f"✅ DataFrame created with {len(df)} rows and {len(df.columns)} columns")

# Create a new CSV filename (don't overwrite the Excel file)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_file = f"justframeit_pricelist_filtered_{timestamp}.csv"
print(f"💾 Saving to CSV file: {csv_file}")
df.to_csv(csv_file, index=False)

print(f"\n✅ Successfully created CSV file: {csv_file}")
print(f"📝 Extracted {len(df)} rows from the Excel file")
print(f"📁 Original Excel file remains unchanged: {excel_file}")
print(f"📋 Column headers: {headers[:5]}...")  # Show first 5 headers


In [13]:
#CREATE PRODUCT PRICE CSV (To be sent to Craft)

import pandas as pd
import openpyxl
from datetime import datetime
import re
import xlwings as xw

# Load the Excel file with xlwings to force calculation
excel_file = "price_export_generated_20251114_131558.xlsx"
print(f"📂 Loading Excel file with xlwings: {excel_file}")

# Open with xlwings to force calculation
app = xw.App(visible=False)
wb_xlwings = app.books.open(excel_file)
ws_xlwings = wb_xlwings.sheets[0]

# Force calculation
print(f"🔄 Forcing Excel calculation...")
wb_xlwings.app.calculate()

# Save the calculated workbook
temp_file = f"temp_calculated_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
wb_xlwings.save(temp_file)
wb_xlwings.close()
app.quit()

print(f"✅ Calculated workbook saved as: {temp_file}")

# Now load with openpyxl
print(f"📂 Loading calculated Excel file with openpyxl: {temp_file}")
wb = openpyxl.load_workbook(temp_file, data_only=True)
ws = wb.worksheets[0]
print(f"✅ Excel file loaded successfully")

# Access the first tab
print(f"📄 Accessing first worksheet: {ws.title}")

# Test reading cells K7 and L7 to verify values
print(f"\n🔍 Testing calculated values for cells K7 and L7:")
k7_cell = ws['K7']
l7_cell = ws['L7']
print(f"   K7 calculated value: {k7_cell.value}")
print(f"   K7 data_type: {k7_cell.data_type}")
print(f"   L7 calculated value: {l7_cell.value}")
print(f"   L7 data_type: {l7_cell.data_type}")

# Headers are in row 6
header_row = 6
start_row = 7  # Data starts from row 7

# Find the last row with data in column A (starting from row 7)
last_row = start_row

print(f"\n🔍 Finding last row with data...")
# Find where column A becomes 0 or empty
for row in range(start_row, ws.max_row + 1):
    if row % 100 == 0:  # Log progress every 100 rows
        print(f"   Checking row {row}...")
    cell_value = ws[f'A{row}'].value
    if cell_value == 0 or cell_value is None or cell_value == '':
        break
    last_row = row

print(f"📊 Extracting data from row {start_row} to row {last_row}")

# Get headers from row 6, but check row 5 for dimension indicators
print(f"📋 Reading headers from row {header_row}...")
headers = []
valid_columns = []  # Track which columns to include
dimension_columns = []  # Track which columns are dimensions
found_first_dimension = False
last_dimension_col = 0
for col in range(1, ws.max_column + 1):
    if col % 50 == 0:  # Log progress every 50 columns
        print(f"   Processing column {col}...")
    # Check row 5 for dimension indicator (e.g., "10.0 x 20.0")
    dimension_cell = ws.cell(row=5, column=col)
    dimension_value = dimension_cell.value
    
    # Get the header from row 6
    header_cell = ws.cell(row=header_row, column=col)
    header_value = header_cell.value
    
    # Stricter check: dimension must contain 'x' AND at least one number
    is_dimension = False
    if dimension_value and isinstance(dimension_value, str):
        # Check if it contains 'x' and at least one digit
        if 'x' in dimension_value.lower() and re.search(r'\d', dimension_value):
            is_dimension = True
            found_first_dimension = True
            last_dimension_col = col
    
    # If we found dimensions before but now stopped finding them, stop processing columns
    if found_first_dimension and not is_dimension:
        print(f"🛑 Stopping at column {col} - no more dimension headers found")
        break
    
    # If row 5 contains a valid dimension indicator, use it
    if is_dimension:
        headers.append(dimension_value)
        valid_columns.append(col)
        dimension_columns.append(len(headers) - 1)  # Track the index of this dimension column
    elif header_value and not found_first_dimension:
        # Only include non-dimension columns if we haven't found dimensions yet
        headers.append(header_value)
        valid_columns.append(col)
    elif not found_first_dimension:
        headers.append(f'Column_{col}')
        valid_columns.append(col)

print(f"✅ Found {len(headers)} columns (last dimension column: {last_dimension_col})")

# Extract data from row 7 onwards to the last valid row
print(f"📥 Extracting data from {last_row - start_row + 1} rows...")
data = []
for row in range(start_row, last_row + 1):
    if (row - start_row) % 50 == 0:  # Log progress every 50 rows
        print(f"   Processing row {row} ({row - start_row + 1}/{last_row - start_row + 1})...")
    row_data = []
    for idx, col in enumerate(valid_columns):  # Only extract data from valid columns
        cell = ws.cell(row=row, column=col)
        # Get the actual value, not the formula
        cell_value = cell.value
        
        # Log K7 and L7 specifically
        if row == 7 and col in [11, 12]:  # K=11, L=12
            col_letter = openpyxl.utils.get_column_letter(col)
            print(f"   📌 Cell {col_letter}{row}: value={cell_value}, type={type(cell_value)}")
        
        # Check if this is a dimension column
        is_dimension_col = idx in dimension_columns
        
        # Handle special cases
        if cell_value is None:
            row_data.append(None)
        elif isinstance(cell_value, str) and cell_value.startswith('!REF'):
            # Handle reference errors
            row_data.append(None)
        elif isinstance(cell_value, (int, float)):
            # Round all numeric values to 2 decimal places
            row_data.append(round(float(cell_value), 2))
        else:
            row_data.append(cell_value)
    data.append(row_data)

# Create DataFrame
print(f"🔄 Creating DataFrame...")
df = pd.DataFrame(data, columns=headers)
print(f"✅ DataFrame created with {len(df)} rows and {len(df.columns)} columns")

# Create a new CSV filename (don't overwrite the Excel file)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_file = f"justframeit_pricelist_filtered_{timestamp}.csv"
print(f"💾 Saving to CSV file: {csv_file}")
df.to_csv(csv_file, index=False)

print(f"\n✅ Successfully created CSV file: {csv_file}")
print(f"📝 Extracted {len(df)} rows from the Excel file")
print(f"📁 Original Excel file remains unchanged: {excel_file}")
print(f"📋 Column headers: {headers[:5]}...")  # Show first 5 headers

# Clean up temporary file
import os
try:
    os.remove(temp_file)
    print(f"🗑️ Cleaned up temporary file: {temp_file}")
except Exception as e:
    print(f"⚠️ Could not remove temporary file: {e}")


📂 Loading Excel file with xlwings: price_export_generated_20251114_131558.xlsx
🔄 Forcing Excel calculation...
✅ Calculated workbook saved as: temp_calculated_20251114_135613.xlsx
📂 Loading calculated Excel file with openpyxl: temp_calculated_20251114_135613.xlsx
✅ Excel file loaded successfully
📄 Accessing first worksheet: Products Prices

🔍 Testing calculated values for cells K7 and L7:
   K7 calculated value: 0.1
   K7 data_type: n
   L7 calculated value: 1.203888888888889
   L7 data_type: n

🔍 Finding last row with data...
   Checking row 100...
   Checking row 200...
   Checking row 300...
   Checking row 400...
   Checking row 500...
   Checking row 600...
   Checking row 700...
   Checking row 800...
   Checking row 900...
   Checking row 1000...
   Checking row 1100...
   Checking row 1200...
   Checking row 1300...
   Checking row 1400...
   Checking row 1500...
   Checking row 1600...
   Checking row 1700...
   Checking row 1800...
   Checking row 1900...
📊 Extracting data fro

In [9]:
#TEST READING K7 AND L7 - 5 DIFFERENT APPROACHES

import pandas as pd
import openpyxl
from openpyxl import load_workbook

# Load the Excel file
excel_file = "price_export_generated_20251114_131558.xlsx"
print(f"📂 Testing file: {excel_file}\n")

# ============================================================================
# APPROACH 1: Load with data_only=True (calculated values only)
# ============================================================================
print("=" * 80)
print("APPROACH 1: Load with data_only=True (calculated values only)")
print("=" * 80)
try:
    wb1 = load_workbook(excel_file, data_only=True)
    ws1 = wb1.worksheets[0]
    k7_val = ws1['K7'].value
    l7_val = ws1['L7'].value
    print(f"✅ K7 value: {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value: {l7_val} (type: {type(l7_val)})")
    wb1.close()
except Exception as e:
    print(f"❌ Error: {e}")
print()

# ============================================================================
# APPROACH 2: Load with data_only=False (formulas, not calculated values)
# ============================================================================
print("=" * 80)
print("APPROACH 2: Load with data_only=False (formulas, not calculated values)")
print("=" * 80)
try:
    wb2 = load_workbook(excel_file, data_only=False)
    ws2 = wb2.worksheets[0]
    k7_cell = ws2['K7']
    l7_cell = ws2['L7']
    print(f"✅ K7 value: {k7_cell.value} (type: {type(k7_cell.value)})")
    print(f"   K7 data_type: {k7_cell.data_type}")
    print(f"✅ L7 value: {l7_cell.value} (type: {type(l7_cell.value)})")
    print(f"   L7 data_type: {l7_cell.data_type}")
    wb2.close()
except Exception as e:
    print(f"❌ Error: {e}")
print()

# ============================================================================
# APPROACH 3: Use pandas read_excel with openpyxl engine
# ============================================================================
print("=" * 80)
print("APPROACH 3: Use pandas read_excel with openpyxl engine")
print("=" * 80)
try:
    # Read starting from row 6 (0-indexed row 5) to get headers from row 6
    df3 = pd.read_excel(excel_file, sheet_name=0, header=5, engine='openpyxl')
    # K7 would be row 0 (first data row after header), column K (index 10)
    # L7 would be row 0, column L (index 11)
    print(f"✅ Column names: {list(df3.columns[:15])}")
    if len(df3) > 0:
        k7_val = df3.iloc[0, 10] if len(df3.columns) > 10 else "N/A"
        l7_val = df3.iloc[0, 11] if len(df3.columns) > 11 else "N/A"
        print(f"✅ K7 value (row 0, col 10): {k7_val} (type: {type(k7_val)})")
        print(f"✅ L7 value (row 0, col 11): {l7_val} (type: {type(l7_val)})")
except Exception as e:
    print(f"❌ Error: {e}")
print()

# ============================================================================
# APPROACH 4: Access by cell coordinates (row, column)
# ============================================================================
print("=" * 80)
print("APPROACH 4: Access by cell coordinates (row=7, column=11 and 12)")
print("=" * 80)
try:
    wb4 = load_workbook(excel_file, data_only=True)
    ws4 = wb4.worksheets[0]
    k7_cell = ws4.cell(row=7, column=11)  # K=11
    l7_cell = ws4.cell(row=7, column=12)  # L=12
    print(f"✅ K7 (row=7, col=11) value: {k7_cell.value} (type: {type(k7_cell.value)})")
    print(f"   K7 data_type: {k7_cell.data_type}")
    print(f"✅ L7 (row=7, col=12) value: {l7_cell.value} (type: {type(l7_cell.value)})")
    print(f"   L7 data_type: {l7_cell.data_type}")
    wb4.close()
except Exception as e:
    print(f"❌ Error: {e}")
print()

# ============================================================================
# APPROACH 5: Save and reload to force calculation
# ============================================================================
print("=" * 80)
print("APPROACH 5: Save and reload to force calculation")
print("=" * 80)
try:
    # First load without data_only
    wb5a = load_workbook(excel_file, data_only=False)
    temp_file = excel_file.replace('.xlsx', '_temp_calculated.xlsx')
    wb5a.save(temp_file)
    wb5a.close()
    print(f"✅ Saved temporary file: {temp_file}")
    
    # Now reload with data_only=True
    wb5b = load_workbook(temp_file, data_only=True)
    ws5b = wb5b.worksheets[0]
    k7_val = ws5b['K7'].value
    l7_val = ws5b['L7'].value
    print(f"✅ K7 value: {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value: {l7_val} (type: {type(l7_val)})")
    wb5b.close()
    
    # Clean up temp file
    import os
    os.remove(temp_file)
    print(f"✅ Cleaned up temporary file")
except Exception as e:
    print(f"❌ Error: {e}")
print()

print("=" * 80)
print("TESTING COMPLETE")
print("=" * 80)


📂 Testing file: price_export_generated_20251114_131558.xlsx

APPROACH 1: Load with data_only=True (calculated values only)
✅ K7 value: None (type: <class 'NoneType'>)
✅ L7 value: None (type: <class 'NoneType'>)

APPROACH 2: Load with data_only=False (formulas, not calculated values)
✅ K7 value: =(Pricelists!$D$3*(-1)/100) (type: <class 'str'>)
   K7 data_type: f
✅ L7 value: <openpyxl.worksheet.formula.ArrayFormula object at 0x0000023301BC8190> (type: <class 'openpyxl.worksheet.formula.ArrayFormula'>)
   L7 data_type: f

APPROACH 3: Use pandas read_excel with openpyxl engine
✅ Column names: ['Name', 'ID', 'Product Code', 'Location Code', 'eCommerce Description', 'Price computation', 'Material Cost per uni', 'Associated Service', 'Associated Work Center', 'Associated Cost per Employee per Hour', 'Margin', 'Column1', 'Column2', 'Column3', 'Column4']
✅ K7 value (row 0, col 10): nan (type: <class 'numpy.float64'>)
✅ L7 value (row 0, col 11): nan (type: <class 'numpy.float64'>)

APPROACH 4: 

In [11]:
#TEST READING K7 AND L7 - APPROACHES 6-10 WITH PROPER LOADING TIME

import pandas as pd
import openpyxl
from openpyxl import load_workbook
import time
import os

# Load the Excel file
excel_file = "price_export_generated_20251114_131558.xlsx"
print(f"📂 Testing file: {excel_file}\n")

# ============================================================================
# APPROACH 6: Load with data_only=True and wait for file to fully load
# ============================================================================
print("=" * 80)
print("APPROACH 6: Load with data_only=True and wait for file to fully load")
print("=" * 80)
try:
    print("⏳ Loading workbook...")
    wb6 = load_workbook(excel_file, data_only=True)
    time.sleep(2)  # Wait 2 seconds for file to fully load
    print("✅ Workbook loaded, accessing worksheet...")
    ws6 = wb6.worksheets[0]
    time.sleep(1)  # Wait 1 second
    print("✅ Worksheet accessed, reading cells...")
    k7_val = ws6['K7'].value
    l7_val = ws6['L7'].value
    print(f"✅ K7 value: {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value: {l7_val} (type: {type(l7_val)})")
    wb6.close()
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 7: Load with data_only=False, wait, then access formulas
# ============================================================================
print("=" * 80)
print("APPROACH 7: Load with data_only=False, wait, then access formulas")
print("=" * 80)
try:
    print("⏳ Loading workbook with formulas...")
    wb7 = load_workbook(excel_file, data_only=False)
    time.sleep(2)  # Wait 2 seconds
    print("✅ Workbook loaded, accessing worksheet...")
    ws7 = wb7.worksheets[0]
    time.sleep(1)  # Wait 1 second
    print("✅ Worksheet accessed, reading cells...")
    k7_cell = ws7['K7']
    l7_cell = ws7['L7']
    print(f"✅ K7 value: {k7_cell.value} (type: {type(k7_cell.value)})")
    print(f"   K7 data_type: {k7_cell.data_type}")
    print(f"✅ L7 value: {l7_cell.value} (type: {type(l7_cell.value)})")
    print(f"   L7 data_type: {l7_cell.data_type}")
    wb7.close()
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 8: Use pandas with wait time
# ============================================================================
print("=" * 80)
print("APPROACH 8: Use pandas read_excel with wait time")
print("=" * 80)
try:
    print("⏳ Reading Excel file with pandas...")
    time.sleep(1)  # Wait before reading
    df8 = pd.read_excel(excel_file, sheet_name=0, header=5, engine='openpyxl')
    time.sleep(2)  # Wait after reading
    print("✅ DataFrame loaded")
    print(f"✅ DataFrame shape: {df8.shape}")
    print(f"✅ Column names (first 15): {list(df8.columns[:15])}")
    if len(df8) > 0 and len(df8.columns) > 11:
        k7_val = df8.iloc[0, 10]
        l7_val = df8.iloc[0, 11]
        print(f"✅ K7 value (row 0, col 10): {k7_val} (type: {type(k7_val)})")
        print(f"✅ L7 value (row 0, col 11): {l7_val} (type: {type(l7_val)})")
    else:
        print("⚠️ DataFrame doesn't have enough rows or columns")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 9: Access by cell coordinates with extended wait
# ============================================================================
print("=" * 80)
print("APPROACH 9: Access by cell coordinates with extended wait")
print("=" * 80)
try:
    print("⏳ Loading workbook...")
    wb9 = load_workbook(excel_file, data_only=True)
    time.sleep(3)  # Wait 3 seconds
    print("✅ Workbook loaded, accessing worksheet...")
    ws9 = wb9.worksheets[0]
    time.sleep(2)  # Wait 2 seconds
    print("✅ Worksheet accessed, reading cells by coordinates...")
    k7_cell = ws9.cell(row=7, column=11)  # K=11
    l7_cell = ws9.cell(row=7, column=12)  # L=12
    time.sleep(1)  # Wait 1 second
    print(f"✅ K7 (row=7, col=11) value: {k7_cell.value} (type: {type(k7_cell.value)})")
    print(f"   K7 data_type: {k7_cell.data_type}")
    print(f"✅ L7 (row=7, col=12) value: {l7_cell.value} (type: {type(l7_cell.value)})")
    print(f"   L7 data_type: {l7_cell.data_type}")
    wb9.close()
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 10: Save, wait, reload with extended delays
# ============================================================================
print("=" * 80)
print("APPROACH 10: Save, wait, reload with extended delays")
print("=" * 80)
try:
    print("⏳ Loading workbook without data_only...")
    wb10a = load_workbook(excel_file, data_only=False)
    time.sleep(2)  # Wait 2 seconds
    temp_file = excel_file.replace('.xlsx', '_temp_calculated.xlsx')
    print(f"⏳ Saving temporary file: {temp_file}")
    wb10a.save(temp_file)
    wb10a.close()
    time.sleep(3)  # Wait 3 seconds after saving
    print(f"✅ Saved and closed, waiting before reload...")
    
    print("⏳ Reloading with data_only=True...")
    wb10b = load_workbook(temp_file, data_only=True)
    time.sleep(3)  # Wait 3 seconds
    print("✅ Workbook reloaded, accessing worksheet...")
    ws10b = wb10b.worksheets[0]
    time.sleep(2)  # Wait 2 seconds
    print("✅ Worksheet accessed, reading cells...")
    k7_val = ws10b['K7'].value
    l7_val = ws10b['L7'].value
    print(f"✅ K7 value: {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value: {l7_val} (type: {type(l7_val)})")
    wb10b.close()
    
    # Clean up temp file
    time.sleep(1)
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"✅ Cleaned up temporary file")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

print("=" * 80)
print("TESTING COMPLETE (APPROACHES 6-10)")
print("=" * 80)


📂 Testing file: price_export_generated_20251114_131558.xlsx

APPROACH 6: Load with data_only=True and wait for file to fully load
⏳ Loading workbook...
✅ Workbook loaded, accessing worksheet...
✅ Worksheet accessed, reading cells...
✅ K7 value: None (type: <class 'NoneType'>)
✅ L7 value: None (type: <class 'NoneType'>)

APPROACH 7: Load with data_only=False, wait, then access formulas
⏳ Loading workbook with formulas...
✅ Workbook loaded, accessing worksheet...
✅ Worksheet accessed, reading cells...
✅ K7 value: =(Pricelists!$D$3*(-1)/100) (type: <class 'str'>)
   K7 data_type: f
✅ L7 value: <openpyxl.worksheet.formula.ArrayFormula object at 0x000002337CE3A5D0> (type: <class 'openpyxl.worksheet.formula.ArrayFormula'>)
   L7 data_type: f

APPROACH 8: Use pandas read_excel with wait time
⏳ Reading Excel file with pandas...
✅ DataFrame loaded
✅ DataFrame shape: (1893, 182)
✅ Column names (first 15): ['Name', 'ID', 'Product Code', 'Location Code', 'eCommerce Description', 'Price computation

In [12]:
# TEST READING K7 AND L7 - APPROACHES 11-15 (Copy/Paste Values)

import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import os
import time
from copy import copy

# Load the Excel file
excel_file = "price_export_generated_20251114_131558.xlsx"
print(f"📂 Testing file: {excel_file}\n")

# ============================================================================
# APPROACH 11: Copy values only to new workbook
# ============================================================================
print("=" * 80)
print("APPROACH 11: Copy values only to new workbook")
print("=" * 80)
try:
    print("⏳ Loading workbook with data_only=True...")
    wb11a = load_workbook(excel_file, data_only=True)
    ws11a = wb11a.worksheets[0]
    
    print("⏳ Creating new workbook and copying values...")
    from openpyxl import Workbook
    wb11b = Workbook()
    ws11b = wb11b.active
    
    # Copy only values from the original sheet
    for row in ws11a.iter_rows():
        for cell in row:
            ws11b[cell.coordinate].value = cell.value
    
    temp_file = excel_file.replace('.xlsx', '_temp_values_only.xlsx')
    print(f"⏳ Saving values-only file: {temp_file}")
    wb11b.save(temp_file)
    wb11a.close()
    wb11b.close()
    
    print("⏳ Reloading values-only file...")
    wb11c = load_workbook(temp_file, data_only=True)
    ws11c = wb11c.worksheets[0]
    k7_val = ws11c['K7'].value
    l7_val = ws11c['L7'].value
    print(f"✅ K7 value: {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value: {l7_val} (type: {type(l7_val)})")
    wb11c.close()
    
    # Clean up
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"✅ Cleaned up temporary file")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 12: Use xlwings to force calculation (if available)
# ============================================================================
print("=" * 80)
print("APPROACH 12: Use xlwings to force calculation")
print("=" * 80)
try:
    import xlwings as xw
    print("⏳ Opening workbook with xlwings...")
    app = xw.App(visible=False)
    wb12 = xw.Book(excel_file)
    ws12 = wb12.sheets[0]
    
    print("⏳ Reading K7 and L7 values...")
    k7_val = ws12.range('K7').value
    l7_val = ws12.range('L7').value
    print(f"✅ K7 value: {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value: {l7_val} (type: {type(l7_val)})")
    
    wb12.close()
    app.quit()
except ImportError:
    print("⚠️ xlwings not installed - skipping this approach")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 13: Read raw XML from Excel file
# ============================================================================
print("=" * 80)
print("APPROACH 13: Read raw XML from Excel file")
print("=" * 80)
try:
    import zipfile
    import xml.etree.ElementTree as ET
    
    print("⏳ Opening Excel file as ZIP...")
    with zipfile.ZipFile(excel_file, 'r') as zip_ref:
        # Read the worksheet XML
        sheet_xml = zip_ref.read('xl/worksheets/sheet1.xml')
        root = ET.fromstring(sheet_xml)
        
        # Define namespace
        ns = {'main': 'http://schemas.openxmlformats.org/spreadsheetml/2006/main'}
        
        print("⏳ Searching for K7 and L7 cells in XML...")
        for cell in root.findall('.//main:c', ns):
            ref = cell.get('r')
            if ref in ['K7', 'L7']:
                value_elem = cell.find('main:v', ns)
                value = value_elem.text if value_elem is not None else None
                print(f"✅ {ref} raw XML value: {value}")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 14: Copy entire sheet to new workbook with paste special values
# ============================================================================
print("=" * 80)
print("APPROACH 14: Copy entire sheet with manual value extraction")
print("=" * 80)
try:
    print("⏳ Loading workbook...")
    wb14a = load_workbook(excel_file, data_only=True)
    ws14a = wb14a.worksheets[0]
    
    print("⏳ Creating new workbook...")
    from openpyxl import Workbook
    wb14b = Workbook()
    ws14b = wb14b.active
    
    print("⏳ Copying all cell values (this may take a moment)...")
    for row_idx, row in enumerate(ws14a.iter_rows(min_row=1, max_row=ws14a.max_row), 1):
        if row_idx % 100 == 0:
            print(f"   Processing row {row_idx}...")
        for col_idx, cell in enumerate(row, 1):
            # Only copy the value, not formulas or formatting
            new_cell = ws14b.cell(row=row_idx, column=col_idx)
            new_cell.value = cell.value
    
    temp_file = excel_file.replace('.xlsx', '_temp_manual_copy.xlsx')
    print(f"⏳ Saving copied workbook: {temp_file}")
    wb14b.save(temp_file)
    wb14a.close()
    wb14b.close()
    
    print("⏳ Reloading copied workbook...")
    wb14c = load_workbook(temp_file, data_only=True)
    ws14c = wb14c.worksheets[0]
    k7_val = ws14c['K7'].value
    l7_val = ws14c['L7'].value
    print(f"✅ K7 value: {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value: {l7_val} (type: {type(l7_val)})")
    wb14c.close()
    
    # Clean up
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"✅ Cleaned up temporary file")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 15: Use pandas to read and write back as values
# ============================================================================
print("=" * 80)
print("APPROACH 15: Use pandas to read and write back as values")
print("=" * 80)
try:
    print("⏳ Reading Excel file with pandas...")
    # Read the entire sheet
    df15 = pd.read_excel(excel_file, sheet_name=0, header=None, engine='openpyxl')
    
    print(f"✅ DataFrame shape: {df15.shape}")
    print("⏳ Accessing K7 and L7 (row 6, columns 10 and 11)...")
    # K7 is row 6 (0-indexed), column 10 (K is the 11th column, 0-indexed = 10)
    # L7 is row 6 (0-indexed), column 11 (L is the 12th column, 0-indexed = 11)
    k7_val = df15.iloc[6, 10] if df15.shape[0] > 6 and df15.shape[1] > 10 else None
    l7_val = df15.iloc[6, 11] if df15.shape[0] > 6 and df15.shape[1] > 11 else None
    print(f"✅ K7 value (row 6, col 10): {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value (row 6, col 11): {l7_val} (type: {type(l7_val)})")
    
    # Write back to new file
    temp_file = excel_file.replace('.xlsx', '_temp_pandas.xlsx')
    print(f"⏳ Writing to temporary file: {temp_file}")
    df15.to_excel(temp_file, index=False, header=False, engine='openpyxl')
    
    print("⏳ Reloading with openpyxl...")
    wb15 = load_workbook(temp_file, data_only=True)
    ws15 = wb15.worksheets[0]
    k7_val_reload = ws15['K7'].value
    l7_val_reload = ws15['L7'].value
    print(f"✅ K7 value (reloaded): {k7_val_reload} (type: {type(k7_val_reload)})")
    print(f"✅ L7 value (reloaded): {l7_val_reload} (type: {type(l7_val_reload)})")
    wb15.close()
    
    # Clean up
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"✅ Cleaned up temporary file")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

print("=" * 80)
print("TESTING COMPLETE (APPROACHES 11-15)")
print("=" * 80)


📂 Testing file: price_export_generated_20251114_131558.xlsx

APPROACH 11: Copy values only to new workbook
⏳ Loading workbook with data_only=True...
⏳ Creating new workbook and copying values...
⏳ Saving values-only file: price_export_generated_20251114_131558_temp_values_only.xlsx
⏳ Reloading values-only file...
✅ K7 value: None (type: <class 'NoneType'>)
✅ L7 value: None (type: <class 'NoneType'>)
✅ Cleaned up temporary file

APPROACH 12: Use xlwings to force calculation
⏳ Opening workbook with xlwings...
⏳ Reading K7 and L7 values...
✅ K7 value: 0.1 (type: <class 'float'>)
✅ L7 value: 1.203888888888889 (type: <class 'float'>)

APPROACH 13: Read raw XML from Excel file
⏳ Opening Excel file as ZIP...
⏳ Searching for K7 and L7 cells in XML...
✅ K7 raw XML value: None
✅ L7 raw XML value: None

APPROACH 14: Copy entire sheet with manual value extraction
⏳ Loading workbook...
⏳ Creating new workbook...
⏳ Copying all cell values (this may take a moment)...
   Processing row 100...
   Proce

In [21]:


# TEST READING K7 AND L7 - APPROACHES 11-15 (Copy/Paste Values)

import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import os
import time
from copy import copy

# Load the Excel file
excel_file = "price_export_generated_20251114_153059.xlsx"
print(f"📂 Testing file: {excel_file}\n")

# ============================================================================
# APPROACH 11: Copy values only to new workbook
# ============================================================================
print("=" * 80)
print("APPROACH 11: Copy values only to new workbook")
print("=" * 80)
try:
    print("⏳ Loading workbook with data_only=True...")
    wb11a = load_workbook(excel_file, data_only=True)
    ws11a = wb11a.worksheets[0]
    
    print("⏳ Creating new workbook and copying values...")
    from openpyxl import Workbook
    wb11b = Workbook()
    ws11b = wb11b.active
    
    # Copy only values from the original sheet
    for row in ws11a.iter_rows():
        for cell in row:
            ws11b[cell.coordinate].value = cell.value
    
    temp_file = excel_file.replace('.xlsx', '_temp_values_only.xlsx')
    print(f"⏳ Saving values-only file: {temp_file}")
    wb11b.save(temp_file)
    wb11a.close()
    wb11b.close()
    
    print("⏳ Reloading values-only file...")
    wb11c = load_workbook(temp_file, data_only=True)
    ws11c = wb11c.worksheets[0]
    k7_val = ws11c['K7'].value
    l7_val = ws11c['L7'].value
    print(f"✅ K7 value: {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value: {l7_val} (type: {type(l7_val)})")
    wb11c.close()
    
    # Clean up
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"✅ Cleaned up temporary file")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 12: Use xlwings to force calculation (if available)
# ============================================================================
print("=" * 80)
print("APPROACH 12: Use xlwings to force calculation")
print("=" * 80)
try:
    import xlwings as xw
    print("⏳ Opening workbook with xlwings...")
    app = xw.App(visible=False)
    wb12 = xw.Book(excel_file)
    ws12 = wb12.sheets[0]
    
    print("⏳ Reading K7 and L7 values...")
    k7_val = ws12.range('K7').value
    l7_val = ws12.range('L7').value
    print(f"✅ K7 value: {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value: {l7_val} (type: {type(l7_val)})")
    
    wb12.close()
    app.quit()
except ImportError:
    print("⚠️ xlwings not installed - skipping this approach")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 13: Read raw XML from Excel file
# ============================================================================
print("=" * 80)
print("APPROACH 13: Read raw XML from Excel file")
print("=" * 80)
try:
    import zipfile
    import xml.etree.ElementTree as ET
    
    print("⏳ Opening Excel file as ZIP...")
    with zipfile.ZipFile(excel_file, 'r') as zip_ref:
        # Read the worksheet XML
        sheet_xml = zip_ref.read('xl/worksheets/sheet1.xml')
        root = ET.fromstring(sheet_xml)
        
        # Define namespace
        ns = {'main': 'http://schemas.openxmlformats.org/spreadsheetml/2006/main'}
        
        print("⏳ Searching for K7 and L7 cells in XML...")
        for cell in root.findall('.//main:c', ns):
            ref = cell.get('r')
            if ref in ['K7', 'L7']:
                value_elem = cell.find('main:v', ns)
                value = value_elem.text if value_elem is not None else None
                print(f"✅ {ref} raw XML value: {value}")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 14: Copy entire sheet to new workbook with paste special values
# ============================================================================
print("=" * 80)
print("APPROACH 14: Copy entire sheet with manual value extraction")
print("=" * 80)
try:
    print("⏳ Loading workbook...")
    wb14a = load_workbook(excel_file, data_only=True)
    ws14a = wb14a.worksheets[0]
    
    print("⏳ Creating new workbook...")
    from openpyxl import Workbook
    wb14b = Workbook()
    ws14b = wb14b.active
    
    print("⏳ Copying all cell values (this may take a moment)...")
    for row_idx, row in enumerate(ws14a.iter_rows(min_row=1, max_row=ws14a.max_row), 1):
        if row_idx % 100 == 0:
            print(f"   Processing row {row_idx}...")
        for col_idx, cell in enumerate(row, 1):
            # Only copy the value, not formulas or formatting
            new_cell = ws14b.cell(row=row_idx, column=col_idx)
            new_cell.value = cell.value
    
    temp_file = excel_file.replace('.xlsx', '_temp_manual_copy.xlsx')
    print(f"⏳ Saving copied workbook: {temp_file}")
    wb14b.save(temp_file)
    wb14a.close()
    wb14b.close()
    
    print("⏳ Reloading copied workbook...")
    wb14c = load_workbook(temp_file, data_only=True)
    ws14c = wb14c.worksheets[0]
    k7_val = ws14c['K7'].value
    l7_val = ws14c['L7'].value
    print(f"✅ K7 value: {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value: {l7_val} (type: {type(l7_val)})")
    wb14c.close()
    
    # Clean up
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"✅ Cleaned up temporary file")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

# ============================================================================
# APPROACH 15: Use pandas to read and write back as values
# ============================================================================
print("=" * 80)
print("APPROACH 15: Use pandas to read and write back as values")
print("=" * 80)
try:
    print("⏳ Reading Excel file with pandas...")
    # Read the entire sheet
    df15 = pd.read_excel(excel_file, sheet_name=0, header=None, engine='openpyxl')
    
    print(f"✅ DataFrame shape: {df15.shape}")
    print("⏳ Accessing K7 and L7 (row 6, columns 10 and 11)...")
    # K7 is row 6 (0-indexed), column 10 (K is the 11th column, 0-indexed = 10)
    # L7 is row 6 (0-indexed), column 11 (L is the 12th column, 0-indexed = 11)
    k7_val = df15.iloc[6, 10] if df15.shape[0] > 6 and df15.shape[1] > 10 else None
    l7_val = df15.iloc[6, 11] if df15.shape[0] > 6 and df15.shape[1] > 11 else None
    print(f"✅ K7 value (row 6, col 10): {k7_val} (type: {type(k7_val)})")
    print(f"✅ L7 value (row 6, col 11): {l7_val} (type: {type(l7_val)})")
    
    # Write back to new file
    temp_file = excel_file.replace('.xlsx', '_temp_pandas.xlsx')
    print(f"⏳ Writing to temporary file: {temp_file}")
    df15.to_excel(temp_file, index=False, header=False, engine='openpyxl')
    
    print("⏳ Reloading with openpyxl...")
    wb15 = load_workbook(temp_file, data_only=True)
    ws15 = wb15.worksheets[0]
    k7_val_reload = ws15['K7'].value
    l7_val_reload = ws15['L7'].value
    print(f"✅ K7 value (reloaded): {k7_val_reload} (type: {type(k7_val_reload)})")
    print(f"✅ L7 value (reloaded): {l7_val_reload} (type: {type(l7_val_reload)})")
    wb15.close()
    
    # Clean up
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"✅ Cleaned up temporary file")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
print()

print("=" * 80)
print("TESTING COMPLETE (APPROACHES 11-15)")
print("=" * 80)


📂 Testing file: price_export_generated_20251114_153059.xlsx

APPROACH 11: Copy values only to new workbook
⏳ Loading workbook with data_only=True...
⏳ Creating new workbook and copying values...
⏳ Saving values-only file: price_export_generated_20251114_153059_temp_values_only.xlsx
⏳ Reloading values-only file...
✅ K7 value: None (type: <class 'NoneType'>)
✅ L7 value: None (type: <class 'NoneType'>)
✅ Cleaned up temporary file

APPROACH 12: Use xlwings to force calculation
⏳ Opening workbook with xlwings...
⏳ Reading K7 and L7 values...
✅ K7 value: 0.1 (type: <class 'float'>)
✅ L7 value: 1.203888888888889 (type: <class 'float'>)

APPROACH 13: Read raw XML from Excel file
⏳ Opening Excel file as ZIP...
⏳ Searching for K7 and L7 cells in XML...
✅ K7 raw XML value: None
✅ L7 raw XML value: None

APPROACH 14: Copy entire sheet with manual value extraction
⏳ Loading workbook...
⏳ Creating new workbook...
⏳ Copying all cell values (this may take a moment)...
   Processing row 100...
   Proce

In [34]:
excel_file = "price_export_generated_20251114_153059.xlsx"

# Save the Excel file as CSV
print("=" * 80)
print("SAVING EXCEL FILE AS CSV (WITH FORMULA EVALUATION)")
print("=" * 80)
try:
    print(f"⏳ Reading Excel file: {excel_file}")
    
    import pandas as pd
    import openpyxl
    import time
    
    csv_file = excel_file.replace('.xlsx', '.csv')
    
    print("⏳ Converting Excel to CSV using openpyxl and pandas...")
    
    # Load workbook with data_only=True to get calculated values instead of formulas
    print("⏳ Loading workbook (this may take a moment)...")
    wb = openpyxl.load_workbook(excel_file, data_only=True)
    ws = wb.active
    
    print(f"✅ Workbook loaded. Active sheet: {ws.title}")
    print(f"📊 Sheet dimensions: {ws.max_row} rows x {ws.max_column} columns")
    
    # Wait 10 seconds to ensure formulas are fully calculated
    print("⏳ Waiting 10 seconds to ensure all formulas are fully loaded and calculated...")
    time.sleep(30)
    print("✅ Wait complete. Proceeding with data extraction...")
    
    # Extract all data as values
    print("⏳ Extracting data from cells...")
    data = []
    for idx, row in enumerate(ws.iter_rows(values_only=True), 1):
        data.append(row)
        if idx % 100 == 0:
            print(f"   Processed {idx} rows...")
    
    print(f"✅ Extracted {len(data)} rows of data")
    
    wb.close()
    
    # Create DataFrame from the extracted values
    print("⏳ Creating DataFrame...")
    df = pd.DataFrame(data)
    
    # Print sample values to verify data extraction
    print("\n📋 Sample values from DataFrame:")
    print(f"   Shape: {df.shape}")
    if df.shape[0] > 6 and df.shape[1] > 11:
        print(f"   K7 (row 6, col 10): {df.iloc[6, 10]}")
        print(f"   L7 (row 6, col 11): {df.iloc[6, 11]}")
    
    # Save to CSV
    print(f"\n⏳ Saving to CSV: {csv_file}")
    df.to_csv(csv_file, index=False, header=False)
    
    print(f"✅ Successfully saved to {csv_file}")
    
    # Read the CSV to get row/column count
    df_check = pd.read_csv(csv_file, header=None)
    print(f"📊 CSV contains {len(df_check)} rows and {len(df_check.columns)} columns")
    
except Exception as e:
    print(f"❌ Error saving to CSV: {e}")
    import traceback
    traceback.print_exc()
print()


SAVING EXCEL FILE AS CSV (WITH FORMULA EVALUATION)
⏳ Reading Excel file: price_export_generated_20251114_153059.xlsx
⏳ Converting Excel to CSV using openpyxl and pandas...
⏳ Loading workbook (this may take a moment)...
✅ Workbook loaded. Active sheet: Products Prices
📊 Sheet dimensions: 2001 rows x 182 columns
⏳ Waiting 10 seconds to ensure all formulas are fully loaded and calculated...
✅ Wait complete. Proceeding with data extraction...
⏳ Extracting data from cells...
   Processed 100 rows...
   Processed 200 rows...
   Processed 300 rows...
   Processed 400 rows...
   Processed 500 rows...
   Processed 600 rows...
   Processed 700 rows...
   Processed 800 rows...
   Processed 900 rows...
   Processed 1000 rows...
   Processed 1100 rows...
   Processed 1200 rows...
   Processed 1300 rows...
   Processed 1400 rows...
   Processed 1500 rows...
   Processed 1600 rows...
   Processed 1700 rows...
   Processed 1800 rows...
   Processed 1900 rows...
   Processed 2000 rows...
✅ Extracted 20

In [38]:
excel_file = "price_export_generated_20251114_153059.xlsx"

# Save the Excel file as CSV
print("=" * 80)
print("SAVING EXCEL FILE AS CSV (WITH FORMULA EVALUATION)")
print("=" * 80)
try:
    print(f"⏳ Reading Excel file: {excel_file}")
    
    from xlsx2csv import Xlsx2csv
    from io import StringIO
    
    csv_file = excel_file.replace('.xlsx', '.csv')
    
    print("⏳ Converting Excel to CSV using xlsx2csv...")
    
    # Use xlsx2csv to convert Excel to CSV
    # This library reads calculated values from Excel files
    print("⏳ Processing Excel file...")
    Xlsx2csv(excel_file, outputencoding="utf-8").convert(csv_file)
    
    print(f"✅ Successfully saved to {csv_file}")
    
    # Read the CSV to get row/column count and verify
    import pandas as pd
    df_check = pd.read_csv(csv_file, header=None)
    print(f"📊 CSV contains {len(df_check)} rows and {len(df_check.columns)} columns")
    
    # Print sample values to verify data extraction
    print("\n📋 Sample values from CSV:")
    print(f"   Shape: {df_check.shape}")
    if df_check.shape[0] > 6 and df_check.shape[1] > 11:
        print(f"   K7 (row 6, col 10): {df_check.iloc[6, 10]}")
        print(f"   L7 (row 6, col 11): {df_check.iloc[6, 11]}")
    
except Exception as e:
    print(f"❌ Error saving to CSV: {e}")
    import traceback
    traceback.print_exc()
print()


SAVING EXCEL FILE AS CSV (WITH FORMULA EVALUATION)
⏳ Reading Excel file: price_export_generated_20251114_153059.xlsx
⏳ Converting Excel to CSV using xlsx2csv...
⏳ Processing Excel file...
✅ Successfully saved to price_export_generated_20251114_153059.csv
📊 CSV contains 2001 rows and 182 columns

📋 Sample values from CSV:
   Shape: (2001, 182)
   K7 (row 6, col 10): nan
   L7 (row 6, col 11): nan



In [19]:
# CLEAN CHATTER FROM x_configuration RECORD ID 1
# CAUTION !!! Are you sure you want to delete this ? 

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo with allow_none=True to handle None values
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)

# Configuration record ID
config_id = 1

print(f"🧹 Cleaning chatter from x_configuration record ID {config_id}...")

# Search for all messages related to this record
message_ids = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB,
    uid,
    JUSTFRAMEIT_ODOO_API_KEY,
    'mail.message',
    'search',
    [[
        ('model', '=', 'x_configuration'),
        ('res_id', '=', config_id)
    ]]
)

print(f"📧 Found {len(message_ids)} messages in chatter")

if message_ids:
    # Delete all messages
    models.execute_kw(
        JUSTFRAMEIT_ODOO_DB,
        uid,
        JUSTFRAMEIT_ODOO_API_KEY,
        'mail.message',
        'unlink',
        [message_ids]
    )
    print(f"✅ Successfully deleted {len(message_ids)} messages from chatter")
else:
    print("ℹ️ No messages found in chatter")

🧹 Cleaning chatter from x_configuration record ID 1...
📧 Found 3 messages in chatter
✅ Successfully deleted 3 messages from chatter


In [4]:
# FETCH ALL ROUTE INFORMATION FOR PRODUCT.TEMPLATE ID 14918

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo with allow_none=True to handle None values
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)

product_id = 14918

print(f"🔍 Fetching route_ids information for product.template ID {product_id}...")

# Read the product template with route_ids field
product_data = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB,
    uid,
    JUSTFRAMEIT_ODOO_API_KEY,
    'product.template',
    'read',
    [[product_id]],
    {'fields': ['name', 'route_ids']}
)

if product_data:
    product = product_data[0]
    print(f"\n📦 Product: {product.get('name')}")
    print(f"🛣️ Route IDs: {product.get('route_ids')}")
    
    route_ids = product.get('route_ids', [])
    
    if route_ids:
        print(f"\n📋 Found {len(route_ids)} route(s). Fetching details...\n")
        
        # Fetch all route details
        routes = models.execute_kw(
            JUSTFRAMEIT_ODOO_DB,
            uid,
            JUSTFRAMEIT_ODOO_API_KEY,
            'stock.route',
            'read',
            [route_ids]
        )
        
        for route in routes:
            print(f"{'='*80}")
            print(f"Route ID: {route.get('id')}")
            print(f"Name: {route.get('name')}")
            print(f"\nAll fields:")
            for key, value in sorted(route.items()):
                print(f"  {key}: {value}")
            print()
    else:
        print("\nℹ️ No routes assigned to this product")
else:
    print(f"❌ Product with ID {product_id} not found")

🔍 Fetching route_ids information for product.template ID 14918...

📦 Product: Finished Product 20251202_160336
🛣️ Route IDs: [1, 6, 4]

📋 Found 3 route(s). Fetching details...

Route ID: 1
Name: Replenish on Order (MTO)

All fields:
  active: True
  categ_ids: []
  company_id: False
  create_date: 2025-08-27 17:32:28
  create_uid: [1, 'OdooBot']
  display_name: Replenish on Order (MTO)
  id: 1
  name: Replenish on Order (MTO)
  package_type_selectable: False
  product_categ_selectable: True
  product_ids: [14912, 14918]
  product_selectable: True
  rule_ids: [3, 5]
  sale_selectable: True
  sequence: 5
  shipping_selectable: False
  supplied_wh_id: False
  supplier_wh_id: False
  warehouse_domain_ids: [1]
  warehouse_ids: []
  warehouse_selectable: False
  write_date: 2025-11-28 11:50:31
  write_uid: [2, 'louisdresse@gmail.com']

Route ID: 6
Name: Buy

All fields:
  active: True
  categ_ids: []
  company_id: False
  create_date: 2025-09-01 09:44:15
  create_uid: [1, 'OdooBot']
  displa

In [1]:
import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo with allow_none=True to handle None values
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)


In [3]:
import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo with allow_none=True to handle None values
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)

# Fetch all order lines for sale orders 211 and 212
sale_order_ids = [211, 212]

for sale_order_id in sale_order_ids:
    # Get the sale order to retrieve order line IDs
    sale_order = models.execute_kw(
        JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'sale.order', 'read',
        [sale_order_id],
        {'fields': ['order_line']}
    )

    if sale_order:
        order_line_ids = sale_order[0].get('order_line', [])
        print(f"📋 Sale Order {sale_order_id} has {len(order_line_ids)} order line(s)")
        print(f"Order Line IDs: {order_line_ids}\n")
        
        if order_line_ids:
            # Fetch all order lines with all their fields
            order_lines = models.execute_kw(
                JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
                'sale.order.line', 'read',
                [order_line_ids]
            )
            
            # Display each order line
            for idx, line in enumerate(order_lines, 1):
                print(f"{'='*80}")
                print(f"📦 Order Line {idx} (ID: {line.get('id')})")
                print(f"{'='*80}")
                print(f"\nAll fields:")
                for key, value in sorted(line.items()):
                    print(f"  {key}: {value}")
                print()
        else:
            print("ℹ️ No order lines found for this sale order")
        
        print(f"\n{'#'*80}\n")
    else:
        print(f"❌ Sale Order with ID {sale_order_id} not found\n")


📋 Sale Order 211 has 1 order line(s)
Order Line IDs: [242]

📦 Order Line 1 (ID: 242)

All fields:
  allowed_uom_ids: [1]
  amount_invoiced: 0.0
  amount_to_invoice: 0.0
  analytic_distribution: False
  analytic_line_ids: []
  analytic_precision: 2
  available_product_document_ids: []
  calendar_booking_ids: []
  calendar_event_id: False
  combo_item_id: False
  company_id: [1, 'The Frame Company']
  company_price_include: tax_excluded
  create_date: 2025-12-18 09:36:16
  create_uid: [2, 'louisdresse@gmail.com']
  currency_id: [125, 'EUR']
  customer_lead: 7.0
  discount: 0.0
  display_name: S00211 - Kader op maat (Mat glas) (Louis Test)
  display_qty_widget: False
  display_type: False
  distribution_analytic_account_ids: []
  extra_tax_data: False
  forecast_expected_date: False
  free_qty_today: 0.0
  id: 242
  invoice_lines: []
  invoice_status: to invoice
  is_configurable_product: True
  is_delivery: False
  is_downpayment: False
  is_expense: False
  is_mto: False
  is_product_ar

In [5]:
import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo with allow_none=True to handle None values
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)

# Fetch all BOMs for product 12060
product_id = 12060
print(f"🔍 Searching for BOMs for product ID: {product_id}\n")

# Search for BOMs associated with this product
bom_ids = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'mrp.bom', 'search',
    [[['product_tmpl_id', '=', product_id]]]
)

if bom_ids:
    print(f"📋 Found {len(bom_ids)} BOM(s) for product {product_id}")
    print(f"BOM IDs: {bom_ids}\n")
    
    # Fetch the 'code' field for all found BOMs
    bom_data = models.execute_kw(
        JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'mrp.bom', 'read',
        [bom_ids],
        {'fields': ['code']}
    )
    
    # Display the code values
    for bom in bom_data:
        print(f"{'='*80}")
        print(f"📦 BOM ID: {bom.get('id')}")
        print(f"{'='*80}")
        print(f"Code: {bom.get('code')}")
        print()
else:
    print(f"❌ No BOMs found for product ID {product_id}")


🔍 Searching for BOMs for product ID: 12060

📋 Found 8 BOM(s) for product 12060
BOM IDs: [585, 586, 587, 588, 589, 590, 591, 592]

📦 BOM ID: 585
Code: Kader op maat (Blinkend glas)

📦 BOM ID: 586
Code: Kader op maat (Geen glas)

📦 BOM ID: 587
Code: Kader op maat (Blinkend plexi glas)

📦 BOM ID: 588
Code: Kader op maat (Artglass AR99)

📦 BOM ID: 589
Code: Kader op maat (Mat plexi glas)

📦 BOM ID: 590
Code: Kader op maat (Artglass AR70)

📦 BOM ID: 591
Code: Kader op maat (Artglass AR92)

📦 BOM ID: 592
Code: Kader op maat (Mat glas)

